<a href="https://colab.research.google.com/github/SebastianLealT/OPTIMIZACION-1/blob/main/Optimizaci%C3%B3n_con_restricciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problema de optimización con restricciones
###Johan Sebastian Leal
###Curso de Optimización I
###Universidad del valle- Cali

Importamos los siguientes paquetes:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

##Función a evaluar y algunos otros cálculos

Definimos nuestra función objetivo, en este caso será:

$$f(x,y)=(x+2y-7)^{2}+(2x+y-5)^{2}$$

la cual es conocida como *Booth Function*.

In [ ]:
def booth(x):
  return ((x[0] + (2*x[1]) -7)**2) + (((2*x[0]) + x[1] - 5)**2)

Será necesario conocer el gradiente, el jacobiano o la Hessiana de la función en cuestión, para ello realizamos algunos cálculos manuales y definimos cada una de estas operaciones. Iniciaremos por definir el gradiente de la función Booth como:

$$\nabla f(x,y) = \langle 2(x+2y-7)+4(2x+y-5), 4(x+2y-7)+2(2x+y-5)\rangle$$

In [ ]:
#Se define el gradiente
def Df(x):
  return np.array([(2*(x[0] + (2*x[1]) -7)) + (4*((2*x[0]) + x[1] - 5)) , (4*(x[0] + (2*x[1]) -7)) + (2*((2*x[0]) + x[1] - 5))])

Por último hacemos el cálculo de la Hessiana y definimos la función en el código:

$$H(x,y)=\begin{bmatrix} 10& 8\\ 8& 10\end{bmatrix}$$

In [ ]:
#Se define la Hessiana
def Hf(x):
  x = np.asarray(x)
  H = np.diag([10 , 10 ])
  H[1,0]= 8
  H[0,1]= 8
  return H

##Conclusiones

##Referencias

1. Heath, M. T. (2018). Scientific Computing: An Introductory Survey, Revised Second Edition. Society for Industrial and Applied Mathematics.
2. Chong, E. K., & Zak, S. H. (2004). An introduction to optimization. John Wiley & Sons.